In [1]:
# 거리 기반으로 반경 1km내 음식점 필터링
from math import sin, cos, sqrt, atan2, radians
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import os

In [2]:
# 예제
data_dir = './data/'
food_path = os.path.join(data_dir, 'food.csv')
place = pd.read_csv(food_path)
place['placeID'] = place.apply(lambda x : x['placeName'] + x['placeAddress'], axis = 1)
place['placeID'] = place['placeID'].apply(lambda x : x.replace(" ", ""))
place['map'] = place[['longitude','latitude']].apply(lambda x: tuple(x.values),axis=1)

In [2]:
R = 6373.0
def distance_from_coord(o_coord, d_coord):
    x1, y1 = radians(o_coord[0]), radians(o_coord[1])
    x2, y2 = radians(d_coord[0]), radians(d_coord[1])
    dlon = x2 - x1
    dlat = y2 - y1
    a = sin(dlat / 2)**2 + cos(y1) * cos(y2) * sin(dlon / 2)**2 
    c = 2 * atan2(sqrt(a), sqrt(1 - a))    
    return R * c

def filtermap(coor,r=1):
    name=[]
    for idx,plc in enumerate(place['map']):
        if distance_from_coord(coor,plc)<=1:
            name.append(place.loc[idx]['placeID'])
    return name

In [8]:
def create_embedding_file() :
    embed_path = os.path.join(data_dir, 'metapath_embeddings')
    with open(embed_path, 'r') as f:
        id2word_len, emb_dimension = f.readline().split()
        id2word = {}
        word2id = {}
        embeddings = []
        idx = 0
        while True :
            z = f.readline()
            if not z :
                break
            z = z.split()
            word = z[0]
            embedding = list(map(float, z[1:]))
            embeddings.append(embedding)
            id2word[idx] = word
            word2id[word] = idx
            idx += 1
    return id2word, word2id, embeddings

In [48]:
id2place, place2id, place_emb = create_embedding_file()
cossim = cosine_similarity(np.array(place_emb))

In [68]:
def get_nearest_cossim(nearest_list, k=5):
    nearest_ids = [place2id[n] for n in nearest_list]
    cossim = cosine_similarity(np.array(place_emb))
    nearest_cossim = cossim[nearest_ids[0], nearest_ids[1:]]
    topk = np.argsort(nearest_cossim)[::-1][:k]
    return [nearest_ids[i] for i in topk]

In [69]:
nearest_list = filtermap((126.9566625, 37.5507711))
topk = get_nearest_cossim(nearest_list)
topk

In [71]:
id2place[4810], id2place[6844], id2place[11856], id2place[8431], id2place[2033]

('싸다김밥애오개역점서울마포구마포대로1821층',
 '일신기사식당서울용산구효창원로218리챠드빌딩',
 '대청마루서울마포구굴레방로9길3',
 '태능숯불갈비서울마포구굴레방로19-1',
 '37.5익스프레스공덕점서울마포구마포대로11길44-19')